In [82]:
import pandas as pd
import sklearn

In [83]:
df = pd.read_csv('dataset_moyennes_matchs.csv')

In [84]:
# suppression de l'ancien idx
df = df.drop(['Unnamed: 0'], axis=1)
# suppression des variables informatives non nécessaires pour la modélisation
df = df.drop(['FTHG','FTAG'],axis=1)
df = df.drop(['match_id','season','date','away_team_id','away_team_name','home_team_id','home_team_name'],axis=1)
# suppression des variables identifiées comme non significatives
df = df.drop(['home_total_tackle','away_total_tackle','home_att_pen_goal','away_att_pen_goal','HY','AY','HR','AR'], axis=1)

In [85]:
cotes = df[['B365H','B365D','B365A','LBH','LBD','LBA','PSH','PSD','PSA','WHH','WHD','WHA','VCH','VCD','VCA','PSCH','PSCD','PSCA']]

In [86]:
target = df['FTR']

data = df.drop(['FTR','B365H','B365D','B365A','LBH','LBD','LBA','PSH','PSD','PSA','WHH','WHD','WHA','VCH','VCD','VCA','PSCH','PSCD','PSCA'],axis=1)

In [87]:
target = target.replace(['H','A','D'],[1,2,3])

In [88]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1425 entries, 0 to 1424
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   home_team_rating               1425 non-null   float64
 1   home_won_contest               1425 non-null   float64
 2   home_possession_percentage     1425 non-null   float64
 3   home_total_throws              1425 non-null   float64
 4   home_blocked_scoring_att       1425 non-null   float64
 5   home_total_scoring_att         1425 non-null   float64
 6   home_aerial_won                1425 non-null   float64
 7   home_aerial_lost               1425 non-null   float64
 8   home_accurate_pass             1425 non-null   float64
 9   home_total_pass                1425 non-null   float64
 10  home_won_corners               1425 non-null   float64
 11  home_shot_off_target           1425 non-null   float64
 12  home_ontarget_scoring_att      1425 non-null   f

In [89]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [90]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [97]:
from sklearn.ensemble import BaggingClassifier
bc = BaggingClassifier(n_estimators=1000, oob_score=True, random_state=321)
bc.fit(X_train_scaled, y_train)
bc.oob_score_

0.5035087719298246

In [98]:
bc.score(X_train_scaled, y_train)

1.0

In [99]:
bc.score(X_test_scaled, y_test)

0.519298245614035

In [100]:
y_pred = bc.predict(X_test_scaled)
pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])

Classe prédite,1,2,3
Classe réelle,,,
1,112,20,3
2,35,33,5
3,46,28,3


In [101]:
from sklearn.metrics import classification_report

In [102]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.58      0.83      0.68       135
           2       0.41      0.45      0.43        73
           3       0.27      0.04      0.07        77

    accuracy                           0.52       285
   macro avg       0.42      0.44      0.39       285
weighted avg       0.45      0.52      0.45       285

